# Задание

# Основы работы с Apache Spark
Изучите теоретическую часть.

*пришлось поставить OpenJDK и Spark локально, из-за ошибок при установке (хотя spark как уоказалось устанавливался корректно)*

In [1]:
# Настраиваем переменные окружения для работы с Apache Spark
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-19"
os.environ["SPARK_HOME"] = "C:/Program Files/spark-3.3.1-bin-hadoop3"
# Находим установку Spark
import findspark
findspark.init()
# Подключаем необходимые модули для работы со Spark из Python
from pyspark.sql import SparkSession
# Создаем сессию Spark на локальном компьютере
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
from zipfile import ZipFile
with ZipFile('C:/Data/ml-100k.zip', 'r') as f:
    f.extractall('./Data/')

## Задача 1
Модернизируйте заготовку заменив все участки ```<put your code here>``` на ваш код для того, что бы:
* вычислять и выводить на экран статистику по числу оценок для каждого фильма
* вычислять и выводить на экран статистику по числу оценок для всех фильмов


Статистика для *каждого* фильма:
```
Marks for film 346: 1 -> 7, 2 -> 10, 3 -> 32, 4 -> 49, 5 -> 28
Marks for film 474: 1 -> 0, 2 -> 6, 3 -> 34, 4 -> 59, 5 -> 95
Marks for film 265: 1 -> 1, 2 -> 13, 3 -> 62, 4 -> 91, 5 -> 60
Marks for film 465: 1 -> 4, 2 -> 8, 3 -> 26, 4 -> 30, 5 -> 17
Marks for film 451: 1 -> 15, 2 -> 31, 3 -> 37, 4 -> 54, 5 -> 33
Marks for film 86: 1 -> 4, 2 -> 10, 3 -> 23, 4 -> 67, 5 -> 46
Marks for film 257: 1 -> 2, 2 -> 28, 3 -> 81, 4 -> 126, 5 -> 66
Marks for film 222: 1 -> 7, 2 -> 30, 3 -> 108, 4 -> 155, 5 -> 65
Marks for film 40: 1 -> 9, 2 -> 9, 3 -> 20, 4 -> 17, 5 -> 2
Marks for film 29: 1 -> 15, 2 -> 34, 3 -> 45, 4 -> 14, 5 -> 6
```
Для *всех* фильмов:
```
Marks for films ALL: 1 -> 6110, 2 -> 34174, 3 -> 27145, 4 -> 11370, 5 -> 21201
```



In [3]:
import collections
rdd = spark.sparkContext.textFile("./Data/ml-100k/u.data")

проверим что все загрузилось

In [4]:
print('The RDD has %s elements, the first one is %s' %(rdd.count(), rdd.take(1)))

The RDD has 100000 elements, the first one is ['196\t242\t3\t881250949']


In [5]:
for x in rdd.take(1):
    print(x)  

196	242	3	881250949


избавимся от разделителей и сгруппируем по ключу

In [6]:
modify_Rdd = rdd.map(lambda x: (x.split('\t')[1], x.split('\t')[2]))
aggPairRDD = modify_Rdd.groupByKey()

In [7]:
def printStat(inp):  
  marks = []
  ind, stat = inp  
  for x in range(1, 6):
    marks.append(stat.get(str(x), 0))
  print(f'Marks for film {ind}: 1 -> {marks[0]}, 2 -> {marks[1]}, 3 -> {marks[2]}, 4 -> {marks[3]}, 5 -> {marks[4]}')

In [8]:
for i in aggPairRDD.mapValues(lambda x: dict(collections.Counter(x))).collect():
  printStat(i)

Marks for film 346: 1 -> 7, 2 -> 10, 3 -> 32, 4 -> 49, 5 -> 28
Marks for film 474: 1 -> 0, 2 -> 6, 3 -> 34, 4 -> 59, 5 -> 95
Marks for film 265: 1 -> 1, 2 -> 13, 3 -> 62, 4 -> 91, 5 -> 60
Marks for film 465: 1 -> 4, 2 -> 8, 3 -> 26, 4 -> 30, 5 -> 17
Marks for film 451: 1 -> 15, 2 -> 31, 3 -> 37, 4 -> 54, 5 -> 33
Marks for film 86: 1 -> 4, 2 -> 10, 3 -> 23, 4 -> 67, 5 -> 46
Marks for film 257: 1 -> 2, 2 -> 28, 3 -> 81, 4 -> 126, 5 -> 66
Marks for film 222: 1 -> 7, 2 -> 30, 3 -> 108, 4 -> 155, 5 -> 65
Marks for film 40: 1 -> 9, 2 -> 9, 3 -> 20, 4 -> 17, 5 -> 2
Marks for film 29: 1 -> 15, 2 -> 34, 3 -> 45, 4 -> 14, 5 -> 6
Marks for film 387: 1 -> 2, 2 -> 11, 3 -> 20, 4 -> 24, 5 -> 8
Marks for film 274: 1 -> 8, 2 -> 13, 3 -> 72, 4 -> 70, 5 -> 27
Marks for film 486: 1 -> 1, 2 -> 2, 3 -> 20, 4 -> 27, 5 -> 14
Marks for film 144: 1 -> 5, 2 -> 12, 3 -> 59, 4 -> 100, 5 -> 67
Marks for film 1: 1 -> 8, 2 -> 27, 3 -> 96, 4 -> 202, 5 -> 119
Marks for film 277: 1 -> 4, 2 -> 8, 3 -> 22, 4 -> 25, 5 -> 

Marks for film 388: 1 -> 5, 2 -> 11, 3 -> 9, 4 -> 2, 5 -> 1
Marks for film 837: 1 -> 0, 2 -> 2, 3 -> 5, 4 -> 11, 5 -> 7
Marks for film 565: 1 -> 3, 2 -> 5, 3 -> 10, 4 -> 4, 5 -> 0
Marks for film 1166: 1 -> 2, 2 -> 1, 3 -> 4, 4 -> 3, 5 -> 2
Marks for film 1427: 1 -> 0, 2 -> 3, 3 -> 1, 4 -> 0, 5 -> 1
Marks for film 1385: 1 -> 1, 2 -> 1, 3 -> 1, 4 -> 2, 5 -> 0
Marks for film 822: 1 -> 0, 2 -> 1, 3 -> 2, 4 -> 1, 5 -> 0
Marks for film 967: 1 -> 0, 2 -> 2, 3 -> 7, 4 -> 2, 5 -> 1
Marks for film 1087: 1 -> 6, 2 -> 2, 3 -> 1, 4 -> 1, 5 -> 0
Marks for film 592: 1 -> 1, 2 -> 0, 3 -> 5, 4 -> 1, 5 -> 2
Marks for film 890: 1 -> 21, 2 -> 10, 3 -> 6, 4 -> 5, 5 -> 1
Marks for film 998: 1 -> 4, 2 -> 4, 3 -> 5, 4 -> 1, 5 -> 2
Marks for film 985: 1 -> 3, 2 -> 3, 3 -> 10, 4 -> 6, 5 -> 0
Marks for film 1231: 1 -> 3, 2 -> 8, 3 -> 8, 4 -> 3, 5 -> 0
Marks for film 669: 1 -> 9, 2 -> 1, 3 -> 2, 4 -> 1, 5 -> 0
Marks for film 1438: 1 -> 2, 2 -> 0, 3 -> 2, 4 -> 1, 5 -> 0
Marks for film 1068: 1 -> 1, 2 -> 1, 3 -> 5,

In [9]:
modify_Rdd_all = rdd.map(lambda x: (x.split('\t')[2], 1))
aggPairRDD_all = modify_Rdd_all.groupByKey()  
dict_all = dict()
for x in aggPairRDD_all.mapValues(lambda x: len(x)).take(10):
  a,b = x
  dict_all[a] = b
printStat(('ALL', dict_all))

Marks for film ALL: 1 -> 6110, 2 -> 11370, 3 -> 27145, 4 -> 34174, 5 -> 21201


## Задача 2
Произведите подсчёт частоты встречаемости слов с использованием ApacheSpark RDD. Ячейка ниже скачивает текст. Вам требуется:
* Очистить текст от знаков препинания и пустых строк
* Перевести в нижний регистр и разделить по пробелам
* Подсчитать наиболее часто встречающиеся символы
* Использовать RDD

Пример вывода:
```
[('и', 2204),
 ('в', 1977),
 ('я', 1252),
 ('не', 1247),
 ('на', 1094),
 ('он', 755),
 ('как', 717),
 ('с', 693),
 ('что', 653),
 ('его', 502)]
```



In [24]:
!wget http://www.lib.ru/INOOLD/BALZAK/shagren.txt_Ascii.txt | iconv -f cp1251
i = 0
with open('shagren.txt_Ascii.txt', encoding="cp1251") as inF, open('shagren.txt_utf8.txt', "w", encoding="utf-8") as outF:
  for line in inF:
    outF.write(line)

--2022-11-28 14:00:20--  http://www.lib.ru/INOOLD/BALZAK/shagren.txt_Ascii.txt
Resolving www.lib.ru (www.lib.ru)... 81.176.66.163
Connecting to www.lib.ru (www.lib.ru)|81.176.66.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: 'shagren.txt_Ascii.txt'

     0K .......... .......... .......... .......... ..........  106K
    50K .......... .......... .......... .......... ..........  460K
   100K .......... .......... .......... .......... ..........  248K
   150K .......... .......... .......... .......... .......... 1,10M
   200K .......... .......... .......... .......... ..........  921K
   250K .......... .......... .......... .......... ..........  881K
   300K .......... .......... .......... .......... ..........  196K
   350K .......... .......... .......... .......... ..........  469K
   400K .......... .......... .......... .......... .......... 1,09M
   450K .......... .......... .......... .......... .......... 

In [25]:
def lower_clean_str(x):
  clean = """!"#$%^&*()_-=,./;':"<>?[]{}+@"""
  lower = x.lower()
  for x in clean:
    lower = lower.replace(x, '')
  return lower
rdd = spark.sparkContext.textFile("shagren.txt_utf8.txt")
rdd = rdd.map(lower_clean_str)
rdd = rdd.flatMap(lambda s: s.split(' ')).filter(lambda x: x!='')
rdd = rdd.map(lambda word: (word, 1)).reduceByKey(lambda x,y: (x+y)).map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0]))
rdd.take(5)

[('й', 2204), ('ч', 1977), ('с', 1252), ('ое', 1247), ('об', 1094)]